In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

import matplotlib.cm as cm
    
import sys 
sys.path.insert(0, '../../src/')

from astropy.table import Table
import os
import warnings; warnings.simplefilter('ignore')
from multiprocessing import Pool
import time

import corner
import mc3


In [7]:
def cumulative(lgMs_1D):
    """

    Args:
        Ms (numpy): 1D mass array
        mass_bins (numpy): mass bins

    Returns:
        CSMF (numpy): counts in each bin
    """

    mass_bins = np.linspace(4,11,45)

    N = np.histogram(lgMs_1D, bins=mass_bins)[0]
    Nsub = np.sum(N)
    stat = Nsub-np.cumsum(N) 
    return np.insert(stat, 0, Nsub) #to add the missing index

def SHMR(lgMh_2D, alpha=1.85, delta=0.2, sigma=0.3):

    """
    Returns: the log stellar mass array with an added dimension corresponding to the random samples
    """

    M_star_a = 10 # these are the anchor points
    M_halo_a = 11.67

    #print("not normalizing for the upscatter and assuming a 2D input array")
    lgMs_2D = alpha*(lgMh_2D-M_halo_a) - delta*(lgMh_2D-M_halo_a)**2 + M_star_a
    scatter = np.random.normal(loc=0, scale=sigma, size=(lgMs_2D.shape))
    return lgMs_2D + scatter

def MODEL(theta):
    
    alpha, delta, sigma = theta

    data = np.load("../../../data/3000_12_8/truth_lgMh.npy")
    lgMs_2D = SHMR(data, alpha, delta, sigma) # will be a 3D array if sigma is non zero
    
    counts = np.apply_along_axis(cumulative, 1, lgMs_2D)
    quant = np.percentile(counts, np.array([5, 50, 95]), axis=0, method="closest_observation") # median and scatter

    S1 = quant[2, 16] - quant[0, 16] #16, 22, 28 corresponds to 6.5, 7, 7.5 Msol
    S2 = quant[2, 22] - quant[0, 22]
    S3 = quant[2, 28] - quant[0, 28]
    N1 = quant[1, 16]
    N2 = quant[1, 22]
    N3 = quant[1, 28]

    model = np.array([N1, N2, N3, S1, S2, S3])

    return model

In [10]:
MODEL([1.2, 0.3, 0.2])

array([14,  6,  1, 13,  7,  4])

In [8]:

# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# Preamble (create a synthetic dataset, in a real scenario you would
# get your dataset from your own data analysis pipeline):

data = np.array([7, 2, 1, 9, 5, 2,])
uncert = np.array([0.372678,  0.45825757, 0.1,  0.74535599, 0.59721576, 0.48989795])
# ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

# Define the modeling function as a callable:
func = MODEL

# List of additional arguments of func (if necessary):
# Array of initial-guess values of fitting parameters:
params = np.array([1, 1, 1])
# Lower and upper boundaries for the MCMC exploration:
pmin = np.array([1, -0.3, 0])
pmax = np.array([ 3,  1.2,  2.5])
# Parameters' stepping behavior:
pstep = np.array([0.1, 0.1, 0.1])

# Parameter prior probability distributions:
prior    = np.array([ 0.0, 0.0, 0.0])
priorlow = np.array([ 0.0, 0.0, 0.0])
priorup  = np.array([ 0.0, 0.0, 0.0])

# Parameter names:
pnames = ['alpha', 'delta', 'sigma']
texnames = [r'$\alpha$', r'$\delta$', r'$\sigma$']

# Sampler algorithm, choose from: 'snooker', 'demc' or 'mrw'.
sampler = 'snooker'

# MCMC setup:
nsamples =  1e4
burnin   =  100
nchains  =  14
ncpu     =  7
thinning =  1

# MCMC initial draw, choose from: 'normal' or 'uniform'
kickoff = 'normal'
# DEMC snooker pre-MCMC sample size:
hsize = 10

# Optimization before MCMC, choose from: 'lm' or 'trf':
leastsq = 'lm'
chisqscale = False

# MCMC Convergence:
grtest = True
grbreak = 1.01
grnmin = 0.3

# Logging:
log = 'MCMC_run.log'

# File outputs:
savefile = 'MCMC_run.npz'
plots = True
theme = 'indigo'
statistics = 'med_central'
rms = True

# Carter & Winn (2009) Wavelet-likelihood method:
wlike = False


In [9]:
# Run the MCMC:
output = mc3.sample(
    data=data, uncert=uncert, func=func, params=params,
    pmin=pmin, pmax=pmax, pstep=pstep,
    pnames=pnames, texnames=texnames,
    prior=prior, priorlow=priorlow, priorup=priorup,
    sampler=sampler, nsamples=nsamples,  nchains=nchains,
    ncpu=ncpu, burnin=burnin, thinning=thinning,
    leastsq=leastsq, chisqscale=chisqscale,
    grtest=grtest, grbreak=grbreak, grnmin=grnmin,
    hsize=hsize, kickoff=kickoff,
    wlike=wlike, log=log,
    plots=plots, theme=theme, statistics=statistics,
    savefile=savefile, rms=rms,
)


::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
  Multi-core Markov-chain Monte Carlo (mc3).
  Version 3.1.2.
  Copyright (c) 2015-2023 Patricio Cubillos and collaborators.
  mc3 is open-source software under the MIT license (see LICENSE).
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

Least-squares best-fitting parameters:
  [1 1 1]

Yippee Ki Yay Monte Carlo!
Start MCMC chains  (Wed Apr 19 15:38:48 2023)

[:         ]  10.0% completed  (Wed Apr 19 15:38:49 2023)
Out-of-bound Trials:
[ 0  9 18]
Best Parameters: (chisq=0.0000)
[1. 1. 1.]

[::        ]  20.0% completed  (Wed Apr 19 15:38:50 2023)
Out-of-bound Trials:
[ 0 14 26]
Best Parameters: (chisq=0.0000)
[1. 1. 1.]
Gelman-Rubin statistics for free parameters:
[       nan 1.20094801 1.12897703]

[:::       ]  30.0% completed  (Wed Apr 19 15:38:51 2023)
Out-of-bound Trials:
[ 0 15 30]
Best Parameters: (chisq=0.0000)
[1. 1. 1.]
Gelman-Rubin statistics for free parameters:
[       nan 1.

LinAlgError: singular matrix

In [ ]:
run = np.load("MCMC_run.npz")

In [ ]:
run["acceptance_rate"]

array(25.04495504)